[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fin-py/chatgpt-api-hands-on/blob/main/docs/models.ipynb)

# Models

LangChainではさまざまなモデルが扱えます。

LLMs

: 入力としてテキスト文字列を受け取り、出力としてテキスト文字列を返します。

Chat Models

: チャットメッセージのリストを入力として受け取り、チャットメッセージを返します。

Text Embedding Models

: テキストを入力とし、浮動小数点数のリストを返します。

## LLMs

`LLM` クラスはLLMのインタフェースです。OpenAIやCohereなどの異なるモデルでも標準的なインタフェースが提供されるよう設計されます。

`OpenAI` クラスはOpenAIのLLMラッパーです。

このモデルを使うにはOpenAIアカウントを作成し、 [API key](https://platform.openai.com/account/api-keys) を生成し、環境変数 `OPENAI_API_KEY` を設定します。

Pythonパッケージをインストールします。

```bash
pip install openai
```

In [1]:
from langchain.llms import OpenAI

openai_llm = OpenAI(model_name="text-ada-001", n=2, best_of=2)
openai_llm("Can you speak japanese?")

'\n\nI can speak japanese.'

`HuggingFaceHub` クラスはHugging Face HubのLLMラッパーです。

> https://huggingface.co/docs/hub/

このモデルを使うにはHugging Faceのアカウントを作成し、 [Access Token](https://huggingface.co/settings/tokens) を生成し、環境変数 `HUGGINGFACEHUB_API_TOKEN` を設定します。

Pythonパッケージをインストールします。


```bash
pip install huggingface_hub
```

In [2]:
from langchain.llms import HuggingFaceHub

hf_llm = HuggingFaceHub(repo_id="gpt2")
hf_llm("Can you speak japanese?")

' Yes, I understand Chinese. My boyfriend, I am here to speak some language in the future. This is for you, your boyfriend. I am going to let your friend take the next step. You two will'

## Chat Models

チャットモデルに1つ以上のメッセージを渡すことで、チャット結果を取得します。現在LangChainではHumanMessage、AIMessage、SystemMessageなどのメッセージがサポートされています。

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)
chat([HumanMessage(content="Translate this sentence from English to Japanese. I have a pen.")])

AIMessage(content='私はペンを持っています。 (Watashi wa pen wo motteimasu.)', additional_kwargs={}, example=False)

OpenAIのチャットモデルは、入力として複数のメッセージをサポートしています。

> https://platform.openai.com/docs/guides/chat/chat-vs-completions

In [4]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Japanese."),
    HumanMessage(content="I have a pen.")
]
chat(messages)

AIMessage(content='私はペンを持っています。(Watashi wa pen wo motte imasu.)', additional_kwargs={}, example=False)

`generate` を使って複数のメッセージに対する補完ができます。

In [5]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to Japanese."),
        HumanMessage(content="I have a pen.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to Japanese."),
        HumanMessage(content="I have an Apple.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text='私はペンを持っています。(Watashi wa pen wo motte imasu.)', generation_info=None, message=AIMessage(content='私はペンを持っています。(Watashi wa pen wo motte imasu.)', additional_kwargs={}, example=False))], [ChatGeneration(text='私はリンゴを持っています。(Watashi wa ringo wo motte imasu.)', generation_info=None, message=AIMessage(content='私はリンゴを持っています。(Watashi wa ringo wo motte imasu.)', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 58, 'completion_tokens': 41, 'total_tokens': 99}, 'model_name': 'gpt-3.5-turbo'})

`llm_output` からトークンの使用量などが確認できます。

In [6]:
result.llm_output

{'token_usage': {'prompt_tokens': 58,
  'completion_tokens': 41,
  'total_tokens': 99},
 'model_name': 'gpt-3.5-turbo'}

### PromptTemplates

テンプレートが利用できます。

In [8]:
template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", output_language="Japanese", text="I have a pen.").to_messages())

AIMessage(content='私はペンを持っています。(Watashi wa pen wo motte imasu.)', additional_kwargs={}, example=False)

`LLMChain` を使って同様な処理ができます。

In [9]:
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="Japanese", text="I have a pen.")

'私はペンを持っています。(Watashi wa pen wo motte imasu.)'

### Streaming

コールバック処理によりストリーミングをサポートしています。

In [10]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)
resp = chat([HumanMessage(content="きのことたけのこの歌を作ってください")])

♪ きのこ、たけのこ、おいしいおいしい
　森の中で採って、食べよう
　きのこ、たけのこ、栄養たっぷり
　体にもいいから、食べよう

　きのこ、たけのこ、色も形も違う
　でも、どれも美味しいから、食べよう
　きのこ、たけのこ、季節によって
　味が変わるから、楽しみだね

　きのこ、たけのこ、食べると元気になる
　体に栄養を与えて、健康になろう
　きのこ、たけのこ、自然の恵み
　大切にして、美味しく食べよう

## Text Embedding Models

`Embedding` クラスはEmbeddingのインタフェースです。OpenAIやCohereなどのプロバイダに対応し、標準的なインタフェースが提供されるよう設計されています。

Embeddingテキストの一部をベクトル表現にします。ベクトル空間にすることで似ているテキストを探したりします。

`OpenAIEmbeddings` を使うには `tiktoken` をインストールします。

```bash
pip install tiktoken
```

In [14]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
text = "This is a test document."
query_result = embeddings.embed_query(text)
doc_result = embeddings.embed_documents([text])

In [17]:
query_result[:5]

[-0.0031265460420399904,
 0.01113363541662693,
 -0.004037691745907068,
 -0.011746617965400219,
 -0.0009935986017808318]

In [21]:
doc_result[0][:5]

[-0.0031265460635144116,
 0.011133635493097378,
 -0.004037691773639618,
 -0.011746618046080886,
 -0.000993598608605281]